In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("Data/kitCapstoneJan24_data.csv", sep=";")

## selecting the country that we want

In [ ]:
selected_country = 'Country A'
#Get rows with for country A

df_country_a = df[df['countryname'] == selected_country]
df_country_a

## building a additional feature for months (1-12)

important for meanbymonth

In [ ]:
df_country_a['month_normalized'] = df_country_a['month'] % 12
df_country_a

df_country_a = df_country_a[['month_normalized','month', 'countryname'] + [col for col in df_country_a.columns if col not in ['month_normalized','month', 'countryname']]]

df_country_a

## select an vehiclegroup

In [ ]:
selcted_vehiclegroup = 'vehiclegroup01'

# delete all columns that are for a diffe

df1 = df_country_a

# create new dataframe with month and all columns containing vehiclegroup01
df2 = df1.filter(regex=selcted_vehiclegroup)
df2.head()

# drop columns in df1 which contain vehiclegroup
df3 = df1.drop(df1.filter(regex='vehiclegroup').columns, axis=1)
df3.head()

# merge df2 and df3 by index
df4 = df2.merge(df3, left_index=True, right_index=True)
df4.head()

# is there a column month in df4?
df4.columns.str.contains('month').any()
# column month should be the first column, second column is countryname
df4 = df4[['month_normalized','month', 'countryname'] + [col for col in df4.columns if col not in ['month_normalized','month', 'countryname']]]

In [ ]:
# drop rows after row 258
#test = df4.drop(df4.index[258:])
test = df4

## build different dataframes for internal and external data

In [ ]:
df_overall = test
df_overall

In [ ]:
# dataframe with first 18 columns (internal data)
df_internal = test.iloc[:, :21]
df_internal.head()

# dataframe with external features
df_external = test.iloc[:, 21:]
df_external = test['']df_external.
df_external.head()

In [ ]:
# dataframe with external features
df_external = test.iloc[:, 21:]
df_external.loc[:, 'month_normalized'] = test['month_normalized']
df_external.head()

In [ ]:
# dataframe with external features
df_external_info = test.iloc[:259,]
df_external_info.head()



# different stuff. nan lists



In [ ]:
# Assuming df_external_info is a DataFrame
nan_columns = df_external_info.isna().any()
nan_counts = df_external_info.isna().sum()


# Create a list of rows
rows = []
for col, count in nan_counts.items():
    if count > 50:
        row = {"name": col, "NaN Count": count}
        rows.append(row)

# Create a DataFrame
nan_info_df = pd.DataFrame(rows, columns=["name", "NaN Count"])

nan_info_df

nan_info_df.to_csv('nan_list.csv')

# Create a list of rows
rows = []
for col, count in nan_counts.items():
    if count <= 50:
        row = {"name": col, "NaN Count": count}
        rows.append(row)

# Create a DataFrame
not_nan_info_df = pd.DataFrame(rows, columns=["name", "NaN Count"])

not_nan_info_df

not_nan_info_df.to_csv('not_nan_list.csv')


# building target values


In [ ]:
country_a_y = y

In [ ]:
country_a_y

In [ ]:
#take every row from 50 to 255 and 
#X = df_internal.iloc[49:255,:-3]
X = df_internal.iloc[49:255, ~df_internal.columns.str.contains('sales_actuals_monthly__vehiclegroup01__orderintake')]
y = df_internal.iloc[49:255, 3]

X_with_target = df_internal.iloc[49:255,:]


In [ ]:
X

## testing wheel

In [ ]:
df_overall

In [ ]:


use_all_features = True
# if false, assign checkup_features here
checkup_features = ['quota__vehiclegroup01__aq'],['sales_flow_monthly__vehiclegroup01__cp8']
input_df = df_overall

predict_future_values = True
if predict_future_values:
    range_predict = len(input_df)
else:
    range_predict = 257

#methods
mean_per_month = True
interpolation_linear = True
fill_zero = True

if use_all_features:
    checkup_features = input_df.columns
for feature in checkup_features:
    print(feature)
    if input_df[feature].isna().any():
        if mean_per_month:
            # add mean per month
            # Fill in missing values using the corresponding month values
            month_means = {}
            for i in range(len(input_df)):
                month = input_df.iloc[i]['month_normalized']
                month_values = input_df.loc[input_df['month_normalized'] == month, feature]
                if not month_values.isnull().values.all():
                    month_means[month] = month_values.mean()

            # Fill in missing values using the corresponding month values
            input_df[feature+'_meanbymonth']=input_df[feature].loc[:].fillna(input_df['month_normalized'].map(month_means))

        if interpolation_linear:
            input_df[feature+'_interpolated_linear']=input_df[feature].interpolate(method='linear', limit_direction = 'both')

        if fill_zero:
            input_df[feature+'_fill_zero']=input_df[feature].fillna(0)
        
        input_df = input_df.drop(columns=feature, inplace=True)
output_df = input_df

In [ ]:
output_df.head()

In [ ]:
output_df.to_csv('model_data_with_own_predictions.csv')

## SHAP

In [ ]:
import xgboost
import shap
from sklearn import metrics
import matplotlib
shap.initjs()

In [ ]:
# Drop countryname column and replace with numerical values
countryMapping = {'Country A': 1, 'Country B': 2, 'Country C': 3, 'Country D': 4}
X['countryNo'] = X['countryname'].map(countryMapping)
X = X.drop('countryname', axis=1)
X.dtypes

In [ ]:
model = xgboost.XGBRegressor().fit(X, y)

print('Accuracy Score of the Model is: ',model.score(X, y))

explainer = shap.Explainer(model)
shap_values = explainer(X)

In [ ]:
shap.summary_plot(shap_values, X)
shap.summary_plot(shap_values, X, plot_type='bar', max_display=10)